## Dependencias

In [26]:
import pandas as pd
import json #JSON = Java Script Object Notation
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,FLOAT,INTEGER,DATE,CHAR
import os

pd.set_option('display.max_columns',None)

## Credenciales

In [3]:
creds = json.load(open('credenciales.json','rb'))

## Crear conexión a base de datos

In [4]:
url = f"mysql+pymysql://{creds['user']}:{creds['password']}@{creds['servidor']}/{creds['base_de_datos']}"
cnx = create_engine(url,encoding='utf8')
cnx = cnx.connect()

In [5]:
cnx.closed

False

## Lectura y limpieza de datos

In [6]:
ruta = '/home/jose/Documentos/bd/nyc_arrests/'

In [9]:
listaArchivos = [os.path.join(ruta,arch) for arch in os.listdir(ruta) if 'csv' in arch]

In [16]:
datos = pd.concat(map(lambda arch:pd.read_csv(arch,
                                              dtype=str,
                                              usecols=['ARREST_KEY',
                                                       'ARREST_DATE',
                                                       'OFNS_DESC',
                                                       'ARREST_BORO',
                                                       'AGE_GROUP',
                                                       'PERP_SEX',
                                                       'PERP_RACE',
                                                       'Latitude',
                                                       'Longitude']),
                                                       listaArchivos),ignore_index=True)

In [17]:
datos.shape

(5227612, 9)

In [19]:
campos_varchar = ['ARREST_KEY','OFNS_DESC','AGE_GROUP','PERP_RACE']

In [36]:
campos = dict(zip(campos_varchar,
                  map(lambda c:VARCHAR(datos[c].map(lambda x:len(x) if type(x)==str else 0).max()),
                                     campos_varchar)))

In [37]:
campos

{'ARREST_KEY': VARCHAR(length=9),
 'OFNS_DESC': VARCHAR(length=43),
 'AGE_GROUP': VARCHAR(length=7),
 'PERP_RACE': VARCHAR(length=30)}

In [38]:
campos.update({'ARREST_BORO':CHAR(1),'PERP_SEX':CHAR(1),'ARREST_DATE':DATE,'Latitude':FLOAT,'Longitude':FLOAT})

In [39]:
campos

{'ARREST_KEY': VARCHAR(length=9),
 'OFNS_DESC': VARCHAR(length=43),
 'AGE_GROUP': VARCHAR(length=7),
 'PERP_RACE': VARCHAR(length=30),
 'ARREST_BORO': CHAR(length=1),
 'PERP_SEX': CHAR(length=1),
 'ARREST_DATE': sqlalchemy.sql.sqltypes.DATE,
 'Latitude': sqlalchemy.sql.sqltypes.FLOAT,
 'Longitude': sqlalchemy.sql.sqltypes.FLOAT}

In [29]:
datos['ARREST_DATE'] = pd.to_datetime(datos['ARREST_DATE'],format='%m/%d/%Y',errors='coerce')

In [30]:
datos.head()

,ARREST_KEY,ARREST_DATE,OFNS_DESC,ARREST_BORO,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude
0,32311380,2007-06-18,DANGEROUS DRUGS,Q,18-24,M,BLACK,NaN,NaN
1,192799737,2019-01-26,SEX CRIMES,M,45-64,M,BLACK,40.800694331000045,-73.941109285999971
2,193260691,2019-02-06,NaN,M,25-44,M,UNKNOWN,40.757839003000072,-73.991212110999982
3,149117452,2016-01-06,RAPE,K,25-44,M,BLACK,40.648650085000035,-73.95033556299995
4,190049060,2018-11-15,RAPE,K,25-44,M,BLACK,40.67458330800008,-73.93022154099998


## Cargar datos a la nube

In [ ]:
datos.to_sql(con=cnx,
             name='historico_ny',
             chunksize=100000,
             if_exists='replace',
             index=False,
             dtype=campos)